In [13]:
import pandas as pd
import random
import numpy as np
from scipy.spatial import distance_matrix
import itertools

In [3]:
def generate_inter_moves(path, dm):
    not_included_ids = list(set([i for i in range(200)]) - set(path))
    inter_moves = []
    for idx, node in enumerate(path):
        for new_node in not_included_ids:
            new_path = path[:idx] + [new_node] + path[idx + 1:]

            new_cost = dm[path[idx - 1]][new_node] + dm[path[(idx + 1) % 100]][new_node]  # two new edges
            new_cost -= (dm[path[idx - 1]][node] + dm[path[(idx + 1) % 100]][node])  # two old edges
            # we want to min new_cost

            inter_moves.append((new_path, new_cost))
    return inter_moves


def generate_intra_moves(path, type_of_change, dm):
    intra_moves = []
    if type_of_change == 'node':
        for idx1, idx2 in all_possible_comb:
            new_path = path[:idx1] + [path[idx2]] + path[idx1 + 1:idx2] + [path[idx1]] + path[idx2 + 1:]
            new_cost = dm[path[idx1]][path[(idx2 + 1) % 100]] + dm[path[idx1]][path[idx2 - 1]]  # new edges for node1
            new_cost += dm[path[idx2]][path[(idx1 + 1) % 100]] + dm[path[idx2]][path[idx1 - 1]]  # new edges for node2
            new_cost -= (dm[path[idx1]][path[idx1 - 1]] + dm[path[idx1]][path[(idx1 + 1) % 100]])  # old edges for node1
            new_cost -= (dm[path[idx2]][path[idx2 - 1]] + dm[path[idx2]][path[(idx2 + 1) % 100]])  # old edges for node2
            intra_moves.append((new_path, new_cost))
    else:  # edge

        for idx1, idx2 in all_possible_comb:
            if (idx2 + 1) % 100 != idx1:
                new_path = path[:idx1] + path[idx1:idx2 + 1][::-1] + path[idx2 + 1:]  # %100 is const.
                #                 new_path = path[:idx1] + path[idx1:idx2][::-1] + path[idx2:] # %100 is const.
                new_cost = dm[path[idx1]][path[(idx2 + 1) % 100]] + dm[path[idx1 - 1]][path[idx2]]  # new edges
                new_cost -= (dm[path[idx1 - 1]][path[idx1]] + dm[path[idx2]][path[(idx2 + 1) % 100]])  # old edges
                intra_moves.append((new_path, new_cost))

    return intra_moves

In [4]:
def generate_dm(df, show=True):
    temp = df[[0, 1]].to_numpy()
    dm = distance_matrix(temp, temp)

    temp = df[2].to_numpy() // 2
    temp = temp * np.ones((200, 200))
    dm = dm + temp + temp.T
    dm = dm // 1

    for i in range(200):
        dm[i][i] = np.inf

    if show:
        df_dist = pd.DataFrame(dm)
        # display(df_dist)
    return dm


def get_random_solution():
    return random.sample([i for i in range(0, 200)], 100)


def calulate_total_cost(path, dm):
    total = 0
    nr = len(path)
    for idx, node in enumerate(path):
        total += dm[node][path[(idx + 1) % nr]]
    return total


def generate_dm_2(df, show=True):
    temp = df[[0, 1]].to_numpy()
    dm = distance_matrix(temp, temp)

#     temp = df[2].to_numpy() // 2
#     temp = temp * np.ones((200, 200))
#     dm = dm + temp + temp.T
#     dm = dm // 1

    for i in range(200):
        dm[i][i] = np.inf

    if show:
        df_dist = pd.DataFrame(dm)
        # display(df_dist)
    return dm

In [184]:
def gen_candieties_moves(tsp, n=10):
    dm_cost = generate_dm(tsp)
    dm = generate_dm_2(tsp)
    
    out_cost = {}
    out_ww_cost = {}
    
    for node in range(dm.shape[0]):
        out_cost[node] = []
        out_ww_cost[node] = []
        for v in range(n):
            idx = np.argmin(dm_cost[node])
            dm_cost[node][idx] = np.inf
            out_cost[node].append(idx)
            
            idx = np.argmin(dm[node])
            dm[node][idx] = np.inf
            out_ww_cost[node].append(idx)
            
    return out_cost, out_ww_cost
gen_candieties_moves(tsp, n=10)

({0: [149, 19, 115, 69, 50, 49, 178, 38, 91, 121],
  1: [177, 75, 41, 189, 137, 174, 199, 152, 130, 39],
  2: [4, 114, 77, 121, 43, 175, 91, 192, 29, 50],
  3: [164, 128, 178, 40, 34, 0, 65, 132, 19, 159],
  4: [114, 77, 43, 2, 121, 91, 192, 175, 29, 50],
  5: [112, 135, 51, 95, 72, 196, 169, 190, 98, 73],
  6: [98, 66, 156, 172, 141, 24, 190, 68, 72, 87],
  7: [62, 74, 163, 93, 182, 146, 32, 180, 155, 195],
  8: [95, 169, 123, 105, 80, 124, 110, 26, 139, 125],
  9: [167, 60, 101, 189, 126, 75, 135, 99, 175, 134],
  10: [132, 128, 113, 36, 37, 181, 185, 55, 85, 74],
  11: [152, 48, 160, 106, 92, 130, 162, 188, 16, 75],
  12: [94, 89, 72, 73, 42, 31, 98, 179, 190, 95],
  13: [26, 8, 125, 48, 119, 52, 92, 106, 169, 95],
  14: [111, 197, 31, 80, 168, 107, 42, 89, 94, 95],
  15: [117, 108, 53, 22, 55, 171, 195, 62, 18, 28],
  16: [48, 152, 11, 130, 92, 119, 189, 75, 106, 188],
  17: [189, 75, 126, 177, 174, 83, 167, 60, 109, 1],
  18: [117, 22, 53, 55, 195, 108, 15, 62, 171, 28],
  19: [0,

In [112]:
import copy

In [199]:
def change_node(path, dm, idx, nearest_node):
    new_path = path.copy()
    new_path[idx] = nearest_node
    new_cost = dm[path[idx - 1]][nearest_node] + dm[path[(idx + 1) % 100]][nearest_node]  # two new edges
    new_cost -= (dm[path[idx - 1]][path[idx]] + dm[path[(idx + 1) % 100]][path[idx]])  # two old edges
    return (new_path, new_cost)
    

In [200]:
def change_edge_next(path, dm, idx, nearest_node_index):
    new_path = path[:idx+1] + path[idx+1:nearest_node_index+1][::-1] + path[nearest_node_index+1:]  # %100 is const.
    new_cost = dm[path[idx]][path[nearest_node_index]] + dm[path[(idx + 1)%100]][path[(nearest_node_index+1)%100]]  # new edges
    new_cost -= (dm[path[idx]][path[(idx+1)%100]] + dm[path[nearest_node_index]][path[(nearest_node_index+1)%100]])  # old edges
    return (new_path, new_cost)
    

In [201]:
def change_edge_before(path, dm, idx, nearest_node_index):
    new_path = path[:idx] + path[idx:nearest_node_index][::-1] + path[nearest_node_index:] 
    new_cost = dm[path[idx]][path[nearest_node_index]] + dm[path[idx - 1]][path[nearest_node_index-1]]  # new edges
    new_cost -= (dm[path[idx]][path[idx-1]] + dm[path[nearest_node_index]][path[nearest_node_index-1]])  # old edges
    return (new_path, new_cost)


In [203]:
def check_edges(path, dm_cost, dm_ww_cost, candieties_edges, candieties_nodes):
    moves = []
    for idx, node in enumerate(path):
        for nearest_edge in candieties_moves[node]:
            if nearest_edge in path: #if node which create this nearest_edge is in the path 

                nearest_node_index = path.index(nearest_edge)
                
                if idx > nearest_node_index:
                    node_idx, nearest_node_index = nearest_node_index, idx
                else:
                    node_idx = idx
                    
                moves.append(change_edge_next(path, dm_ww_cost, node_idx, nearest_node_index))               
                moves.append(change_edge_before(path, dm_ww_cost, node_idx, nearest_node_index))
            
                
        for nearest_node in candieties_nodes[node]:
            if nearest_node not in path:
                moves.append(change_node(path, dm_cost, idx, nearest_node))
                

    return moves 
dm_cost = generate_dm(tsp)
dm_ww_cost = generate_dm_2(tsp)
path = get_random_solution()
candieties_edges, candieties_nodes = gen_candieties_moves(tsp, n=10)
a = check_edges(path, dm_cost, dm_ww_cost, candieties_edges, candieties_nodes)

In [215]:
def a():
    tsp = pd.read_csv('TSPC.csv', sep=';', header=None)

    dm_cost = generate_dm(tsp)
    dm_ww_cost = generate_dm_2(tsp)
    path = get_random_solution()
    candieties_edges, candieties_nodes = gen_candieties_moves(tsp, n=10)

    change = True

    while change:

        change = False

        possible_moves = check_edges(path, dm_cost, dm_ww_cost, candieties_edges, candieties_nodes)

        possible_moves.sort(key=lambda x: x[1])
    #     print([i for x, i in possible_moves])
    #     break
        if possible_moves[0][1] < 0:
    #         print(possible_moves[0][1])
            path = possible_moves[0][0]
            change = True
#             print(calulate_total_cost(path, dm_cost))

    return path, calulate_total_cost(path, dm_cost)

In [217]:
for i in range(10):
    print(i)
    a()     

0
1
2
3
4
5
6
7
8
9


In [211]:
len(set(path))

100